# Day 7 
## Part 1
I'm going to build a class for a hand that will allow ordering. First build some of the necessary infrastructure. An ordered HandType will be useful, as will a function to give the type of a hand. The code here is possibly too terse, but it gives the counts for blocks of common cards, so e.g. a full house "QQQ66" will look like `{3: 1, 2: 1}`. It's then fairly simple to define and check the requirements for each type of hand.

In [1]:
from enum import IntEnum
from collections import Counter

HandType = IntEnum(
    "HandType", 
    [
        "HIGH_CARD",
        "ONE_PAIR", 
        "TWO_PAIR", 
        "THREE_OF_A_KIND",
        "FULL_HOUSE",
        "FOUR_OF_A_KIND",
        "FIVE_OF_A_KIND"
    ]
)

def value_hand(s: str) -> HandType:
    c = Counter(Counter(s).values())
    for ht, needed in zip(
        reversed(HandType),
        [
            {5: 1}, 
            {4: 1}, 
            {3: 1, 2: 1}, 
            {3: 1}, 
            {2: 2}, 
            {2: 1}, 
            {1: 5}
        ],
    ):
        if all(
            c[card_count] == needed[card_count] 
            for card_count in needed
        ):
            return ht

def parse_data(s):
    return [
        (spl[0], int(spl[1]))
        for spl in [
            line.split() 
            for line in s.strip().splitlines()
        ]
    ]

test_data = parse_data("""32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""")

[ (s, value_hand(s)) for s, _ in test_data ]

[('32T3K', <HandType.ONE_PAIR: 2>),
 ('T55J5', <HandType.THREE_OF_A_KIND: 4>),
 ('KK677', <HandType.TWO_PAIR: 3>),
 ('KTJJT', <HandType.TWO_PAIR: 3>),
 ('QQQJA', <HandType.THREE_OF_A_KIND: 4>)]

Provide the ranking of cards as a constant lookup.

In [2]:
CARDS = { 
    c: v 
    for c, v in zip(
        "AKQJT98765432",
        range(14, 1, -1)
    )
}

CARDS

{'A': 14,
 'K': 13,
 'Q': 12,
 'J': 11,
 'T': 10,
 '9': 9,
 '8': 8,
 '7': 7,
 '6': 6,
 '5': 5,
 '4': 4,
 '3': 3,
 '2': 2}

Finally the class for the ordered hands.

In [3]:
class Hand:
    def __init__(self, s: str):
        self.hand = s
        self.type = value_hand(s)
        
    def __lt__(self, other):
        if self.type != other.type:
            return self.type < other.type
        return (
            [CARDS[c] for c in self.hand] 
            < [CARDS[c] for c in other.hand]
        )
    
    def __repr__(self):
        return f"Hand({self.hand})"
    
sorted(Hand(s) for s, _ in test_data)

[Hand(32T3K), Hand(KTJJT), Hand(KK677), Hand(T55J5), Hand(QQQJA)]

In [4]:
import itertools

def part_1(data):
    sorted_hands = sorted((Hand(hand), bid) for hand, bid in data)
    return sum(
        bid * rank
        for (_, bid), rank
        in zip(sorted_hands, itertools.count(1))
    )

assert part_1(test_data) == 6440

In [5]:
part_1(test_data)

6440

In [6]:
data = parse_data(open("input").read())

part_1(data)

253933213

## Part 2
Amend the cards lookup for the Joker.

In [7]:
CARDS = { 
    c: v 
    for c, v in zip(
        "AKQT98765432J",
        range(14, 1, -1)
    )
}

The Joker cards can be added to the most common other card, and this will always give the highest ranked hand in the absence of flushes.

In [8]:
def value_hand(s: str) -> HandType:
    card_counts = Counter(s)
    joker_count = card_counts["J"]
    non_joker_count = Counter(''.join(c) for c in s if c != "J")
    if non_joker_count:
        top_card = non_joker_count.most_common(1)[0][0]
    else: # all cards are Jokers
        return HandType.FIVE_OF_A_KIND
    non_joker_count[top_card] += joker_count
    c = Counter(non_joker_count.values())
    for ht, needed in zip(
        reversed(HandType),
        [
            {5: 1}, 
            {4: 1}, 
            {3: 1, 2: 1}, 
            {3: 1}, 
            {2: 2}, 
            {2: 1}, 
            {1: 5}
        ]
    ):
        if all(
            c[card_count] == needed[card_count] 
            for card_count in needed
        ):
            return ht
        
assert part_1(test_data) == 5905

In [9]:
part_1(data)

253473930